In [26]:
import numpy as np
from matplotlib import pyplot as plt
from astropy.io import ascii

#To read in files with flux and template flux: each file is one galaxy, currently reads in the SpARCS0035 cluster, can modify for others
#fluxpath = r'''C:\Users\Anna\PHOTOMETRY\BESTFIT_SEDs\EAZY\for_greg\'''
fluxpath = '/Users/grudnick/Work/GOGREEN/Data/Releases/v1.1/PHOTOMETRY/BESTFIT_SEDs/EAZY/for_greg/'
#Example file name: SpARCS-0035_totalall_Ks_1.obs_sed  - number varies from 1-4889 for SpARCS-0035
flux_dat_all = {} #initialize empty dictionary
for n in range(4881, 4889):
    name = "SpARCS-0035_totalall_Ks_" + str(n) + ".obs_sed" #Should build the file name from path to file name
    imstr = fluxpath + name
    flux_dat = ascii.read(imstr) #will overwrite each time
    #flux_dat_all[str(n)] = flux_dat #appends dictonary with each file, key is the number n associated with that file, same as the number in the file name
    #flux_dat_all[str(n)]['lambda'] = np.array([])
    flux_dat_all[str(n)] = {}
    flux_dat_all[str(n)]['lambda'] = np.array(flux_dat['lambda'])
    flux_dat_all[str(n)]['flux_cat'] = np.array(flux_dat['flux_cat'])
    #do for all other columns
    #print (imstr)

print(flux_dat_all['4881']) #prints one galaxy 
flux_dat_all['4881']['lambda'] #prints one galaxy 
#print(flux_dat_all) #prints all galaxies


#print(flux_dat[0])    
#flux_dat = ascii.read(imstr)
#print(flux_dat[0][1]) #k band flux
#print(flux_dat[0][5]) #template k band flux


{'lambda': array([21485.08 ,  3749.196,  4326.617,  5478.757,  6485.849,  8215.966,
        9179.498, 10551.44 , 12583.92 , 35634.26 , 45110.19 , 57593.12 ,
       79594.23 ]), 'flux_cat': array([17.671    ,  0.7336996,  1.985197 ,  1.63024  ,  2.199269 ,
        3.220228 ,  4.764473 ,  4.804726 ,  7.587662 , 24.9403   ,
       28.53927  , 29.99752  , 75.03314  ])}


array([21485.08 ,  3749.196,  4326.617,  5478.757,  6485.849,  8215.966,
        9179.498, 10551.44 , 12583.92 , 35634.26 , 45110.19 , 57593.12 ,
       79594.23 ])

In [27]:
#To read in spectroscopic redshift:
#redpath = r'''C:\Users\Anna\PHOTOMETRY\SPECZ_MATCHED\'''
redpath = '/Users/grudnick/Work/GOGREEN/Data/Releases/v1.1/PHOTOMETRY/SPECZ_MATCHED/'
filename = redpath + "compilation_SpARCS-0035.dat"
red_dat = ascii.read(filename)
#print(red_dat[1][:]) #prints all redshifts
print(red_dat[4881]['spec_z']) #prints redshift for galaxy 4881

#To append flux_dat_all with redshifts from the above file:

flux_dat_all['4881']['zspec'] = red_dat[4881]['spec_z']
#print(flux_dat_all['zspec'])


-99.0
